# Test om koden er riktig implementert

Her er et forslag til testfunksjoner for å sjekke om koden er riktig implementert.
```assert variabel``` vil gi en feilmelding med mindre variabelen ```variabel = True```. For eksempel vil ```assert a == b``` gi en feilmelding med mindre ```a``` og ```b``` er like.

In [1]:
#For eksempel:
variable = True
assert variable, "You need to change 'variable' to True"

In [2]:
from layers import *
from neural_network import NeuralNetwork
from utils import onehot
import numpy as np

In [3]:
#We choose some arbitrary values for the dimensions
b = 1
n_max = 1
m = 2
n = 1

d = 2
k = 2
p = 2

r = 3
L_1 = 1

#Create an arbitrary dataset
x = np.random.randint(0, m, (b,n))
y = np.random.randint(0, m, (b,n_max))

#initialize the layers
feed_forward = FeedForward(d,p)
self_attention = SelfAttention(d,k)
embed_pos = EmbedPosition(n_max,m,d)
un_embed = LinearLayer(d,m, False)
softmax = Softmax()
transformerblock = TransformerBlock(d,k,p)


#a manual forward pass
X = onehot(x, m)  # (b, m, n)
z0 = embed_pos.forward(X)
z1 = feed_forward.forward(z0)
z2 = self_attention.forward(z1)
z3 = un_embed.forward(z2)
Z = softmax.forward(z3) 



#check the shapes
assert X.shape == (b,m,n), f"X.shape={X.shape}, expected {(b,m,n)}"
assert z0.shape == (b,d,n), f"z0.shape={z0.shape}, expected {(b,d,n)}"
assert z1.shape == (b,d,n), f"z1.shape={z1.shape}, expected {(b,d,n)}"
assert z2.shape == (b,d,n), f"z2.shape={z2.shape}, expected {(b,d,n)}"
assert z3.shape == (b,m,n), f"z3.shape={z3.shape}, expected {(b,m,n)}"
assert Z.shape == (b,m,n), f"Z.shape={Z.shape}, expected {(b,m,n)}"

#is X one-hot?
assert X.sum() == b*n, f"X.sum()={X.sum()}, expected {b*n}"


assert np.allclose(Z.sum(axis=1), 1), f"Z.sum(axis=1)={Z.sum(axis=1)}, expected {np.ones(b)}"
assert np.abs(Z.sum() - b*n) < 1e-5, f"Z.sum()={Z.sum()}, expected {b*n}"
assert np.all(Z>=0), f"Z={Z}, expected all entries to be non-negative"


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
<string>:3: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'A', False, aligned=True), Array(float64, 2, 'C', False, aligned=True))


In [7]:
  # y_pred: (batch, m, n)
        # y_true: (batch, n)
        # m = number of classes


#test the forward pass
x = np.random.randint(0, m, (b,n_max))
X = onehot(x, m)
print(X.shape)

#we test with a y that is shorter than the maximum length
n_y = n_max - 1
y = np.random.randint(0, m, (b,n_y))

#initialize a neural network based on the layers above
network = NeuralNetwork(r, d, m, L_1, p, k)

#and a loss function
loss = CrossEntropy()

#do a forward pass
Z = network.forward(X)
print(Z.shape)

#compute the loss
print(Z)
print(y.shape)
L = loss.forward(Z, y)

#get the derivative of the loss wrt Z
grad_Z = loss.backward()

#and perform a backward pass
_ = network.backward()

#and and do a gradient descent step
_ = network.step_gd(0.01)


(1, 2, 1)
(1, 2, 1)
[[[0.62194674]
  [0.37805325]]]
(1, 0)


/Users/hanna/Library/CloudStorage/OneDrive-NTNU/TMA4320 - Vitber/Vitber-indmat-5/layers.py:449: RuntimeWarning: Mean of empty slice.
  per_sequence_loss = per_token_loss.mean(axis=1)
/Users/hanna/micromamba/envs/3.12/lib/python3.12/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m- Resolution failure for literal arguments:
[1mmissing a required argument: 'grad'[0m
[0m[1m- Resolution failure for non-literal arguments:
[1mNone[0m
[0m[0m
[0m[1mDuring: resolving callee type: BoundFunction((<class 'numba.core.types.misc.ClassInstanceType'>, 'backward') for instance.jitclass.NeuralNetwork#117c04980<embedding:instance.jitclass.EmbedPosition#118d0f5c0<embed:instance.jitclass.LinearLayer#118b5cbc0<params:DictType[unicode_type,DictType[unicode_type,array(float64, 2d, A)]<iv=None>]<iv=None>,x:array(float64, 3d, A),has_bias:bool>,w:array(float64, 2d, A),params:DictType[unicode_type,DictType[unicode_type,array(float64, 2d, A)]<iv=None>]<iv=None>>,transformer_blocks:list(instance.jitclass.TransformerBlock#118d0f050<self_attention:instance.jitclass.SelfAttention#118cd3110<W_q:instance.jitclass.LinearLayer#118b5cbc0<params:DictType[unicode_type,DictType[unicode_type,array(float64,

In [ ]:
"""
Here you may add additional tests to for example:

- Check if the ['d'] keys in the parameter dictionaries are not None, or receive something when running backward pass
- Check if the parameters change when you perform a gradient descent step
- Check if the loss decreases when you perform a gradient descent step

This is voluntary, but could be useful.
"""

"\nHere you may add additional tests to for example:\n\n- Check if the ['d'] keys in the parameter dictionaries are not None, or receive something when running backward pass\n- Check if the parameters change when you perform a gradient descent step\n- Check if the loss decreases when you perform a gradient descent step\n\nThis is voluntary, but could be useful.\n"

In [6]:
#check if loss is non-negative
assert L >= 0, f"L={L}, expected L>=0"
assert grad_Z.shape == Z.shape, f"grad_Z.shape={grad_Z.shape}, expected {Z.shape}"

#check if onehot(y) gives zero loss
Y = onehot(y, m)
L = loss.forward(Y, y)
assert L < 1e-5, f"L={L}, expected L<1e-5"


AssertionError: L=[nan], expected L>=0